In [3]:
# Assumptions
## Assumption 1: Each node has a unique addresses that can be used for lexicographical sorting.
## Assumption 2: Each participating peer / node / actor has a full view of the network (i.e. list of all unique addresses is available to each node)

# Definitions
## Definition 1: Layer: the inverse of a tree height (max at the root and 0 at a leaf)
## Definition 2: Max # of layers: tree height 
## Definition 3: Global Address Book: list of all unique addresses in the network
## Definition 4: Partial Address Book: partial list of all unique addresses nodes in the network (either due to algorithm or due to lack of information)

# Structure
## RainTree: A ternary tree where one of the children of each non-leaf node is a copy of itself with a different subtree
## AddrBook: A sorted list of unique addresses that a node / peer is operating on at a given point in time in the algorithm

# Params
## Let X be the % used to calculate the index of the 1st message sent by a node in `AddrBook` defined above (e.g. X = 1/3 when AddrBook has 9 nodes implies the 3rd node in the sorted list)
## Let Y be the % used to calculate the index of the 2nd message sent by a node in `AddrBook` defined above (e.g. Y = 2/3 when AddrBook has 9 nodes implies the 6th node in the sorted list)
## Let Z be the % by which the `AddrBook` defined above is shrunk with each message propagation (e.g. Z = 2/3 when AddrBook has 9 nodes implies a new AddrBook including nodes 1 through 6)


# Questions to answer:
# Q: What should the stopping condition be aside from cheap log3(N)?
# Q: Why does log3 base achieve the desired height?
# Q: Do X & Y need to be constants?
# Q: Can X & Y be random but in some range?
# Q: Does Y need to equal Z?

# Analyze:
# - Run simulation over different N and check if log3(N) always achieves the desired height
# - Increase # simulations and check if the average number of messages different

# TODO:
# - Redundancy & failure

from collections import defaultdict
from collections import deque
from pptree import Node, print_tree

import math
import random
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Helpers
def shrink_list(l, i1, i2):
    if i1 <= i2:
        return l[i1:i2]
    return l[i1:] + l[:i2]

def get_msg_sent(l, i, t):
    s = "[ "
    for idx, n in enumerate(l):
        if idx == i:
            s += f"({n}), "
        elif idx == t:
            s += f"**{n}**, "
        else:
            s += f"{n}, "
    return f"{s[:-2]} ]"

def agg_dicts(d1, d2):
    return {k: d1.get(k, 0) + d2.get(k, 0) for k in set(d1) | set(d2)}

def run_simulations(S, N, X, Y, Z, should_print): # S = num simulations, N = num nodes, X = 1st message, Y = 2nd message, Z = shrinkage
    # Reset global params for simulation
    global global_msg_send_counter
    global global_set_nodes_reached
    global global_map_msg_rec_counter
    global global_map_msg_send_counter
    global global_map_depth_reached_counter
    global global_prop_queue
    global global_missing_nodes

    # Flags are used for visualization (not proving max depth)
    global global_enforce_full_prop
    global global_max_allowed_depth

    # Core logic
    def prop(addr, book, depth, X, Y, Z, node, sender): # addr => curr_add, book => curr_addr_book, depth => curr_depth
        global global_msg_send_counter
        global global_set_nodes_reached
        global global_map_msg_rec_counter
        global global_map_msg_send_counter
        global global_map_depth_reached_counter
        global global_prop_queue
        global global_missing_nodes

        # Flags are used for visualization (not proving max depth)
        global global_enforce_full_prop
        global global_max_allowed_depth

        if len(book) == 0:
            return

        # if depth >= global_max_allowed_depth:
        #     global_map_depth_reached_counter[depth] += 1
        #     print("Max depth reached")
        #     return

        # Add node to the tree
        # node = Node(addr) if parent is None else Node(addr, parent)


        if len(global_missing_nodes) == 0:
            global_map_depth_reached_counter[depth] += 1
            if not global_enforce_full_prop or depth >= global_max_allowed_depth:
                return

        # A network message was made - track it
        global_missing_nodes.discard(addr)
        global_set_nodes_reached.add(addr)
        if addr != sender:
            global_map_msg_rec_counter[addr] += 1

        if should_print:
            print('-----') # Separator to make reading easier

        n = len(book)
        i = book.index(addr)
        # x = (i + math.ceil(n * X)) % n
        # y = (i + math.ceil(n * Y)) % n
        # z = (i + math.ceil(n * Z)) % n
        x = (i + int(n * X)) % n
        y = (i + int(n * Y)) % n
        z = (i + int(n * Z)) % n        

        x_addr = book[x]
        y_addr = book[y]
        
        if x_addr == y_addr:
            y_addr = None
        if x_addr == addr:
            x_addr = None

        # print(f"Global missing nodes ({len(global_missing_nodes)}): ", global_missing_nodes)

        # Send to first target
        if x_addr is not None:
            global_msg_send_counter += 1
            x_book = book.copy()
            x_z = (x  + int(n * Z)) % n
            # x_z = (x  + math.ceil(n * Z)) % n
            x_book_s = shrink_list(x_book, x, x_z)
            global_prop_queue.append((x_addr, x_book_s, depth + 1, X, Y, Z, Node(x_addr, node), addr))
        
            # Assumes successfull propagation
            global_missing_nodes.discard(x_addr)
            global_set_nodes_reached.add(x_addr)

            global_map_msg_send_counter[addr] += 1
            if should_print:
                print(f"Msg 1: {get_msg_sent(book, i, x)}")

        # Send to second target
        if y_addr is not None:
            global_msg_send_counter += 1
            y_book = book.copy()
            y_z = (y  + int(n * Z)) % n
            # y_z = (y  + math.ceil(n * Z)) % n
            y_book_s = shrink_list(y_book, y, y_z)
            global_prop_queue.append((y_addr, y_book_s, depth + 1, X, Y, Z, Node(y_addr, node), addr))
            global_map_msg_send_counter[addr] += 1
            if should_print:
                print(f"Msg 2: {get_msg_sent(book, i, y)}")

            # Assumes successfull propagation
            global_missing_nodes.discard(y_addr)
            global_set_nodes_reached.add(y_addr)

        # This is a demote (not a send) so we do not increment `global_msg_send_counter`
        book_s = shrink_list(book, i, z)
        if len(book_s) > 1:
            global_prop_queue.append((addr, book_s, depth + 1, X, Y, Z, Node(addr, node), addr))

        # return node

    ## Simulations counters
    msg_send_counter_acc = 0
    map_msg_rec_counter_acc = defaultdict(int)
    depth_counter_acc = defaultdict(int)


    # global_addr_book = sorted([chr(ord('A') + i) for i in range(N)])
    global_addr_book = sorted([f'val_{i+1}' for i in range(N)], key=lambda x: int(x.split('_')[1]))


    # Flags are used for visualization (not proving max depth)
    global_enforce_full_prop  = True
    global_max_allowed_depth = math.log(N, 3)
    print("!!!!!!!!", global_max_allowed_depth)

    for _ in range(S):
        # Reset global params for simulation
        global_msg_send_counter = 0
        global_set_nodes_reached = set()
        global_map_msg_rec_counter = defaultdict(int)
        global_map_msg_send_counter = defaultdict(int)
        global_map_depth_reached_counter = defaultdict(int)
        global_prop_queue = deque()
        global_missing_nodes = set(global_addr_book)

        # Start simulation
        orig_addr = 'val_1'
        # orig_addr = 'O'
        # orig_addr = random.choice(global_addr_book)
        # orig_addr = global_addr_book[0]
        root = Node(orig_addr)
        global_prop_queue.append((orig_addr, global_addr_book, 0, X, Y, Z, root, orig_addr))
        # root = None
        while len(global_prop_queue) > 0:
            prop(*global_prop_queue.popleft())
            # node = prop(*global_prop_queue.popleft())
            # root = node if root is None else root

        # Print results
        if should_print:
            print('###################')
            print_tree(root, horizontal=False)    
            print(f"Target Coverage: {Y}")
            print(f"Num nodes: {N}")
            print(f"Global Send Counter: {global_msg_send_counter}")
            print(f"Global Set Reached: {sorted(list(global_set_nodes_reached))}")
            print(f"Global # Times Received: {dict(dict(sorted(global_map_msg_rec_counter.items(), key=lambda item: -item[1])))}")
            print(f"Nodes not reached: {global_set_nodes_reached.difference(global_addr_book)}")

        # Aggregate results
        depth_counter_acc = agg_dicts(depth_counter_acc, global_map_depth_reached_counter)
        msg_send_counter_acc += global_msg_send_counter
        map_msg_rec_counter_acc = agg_dicts(map_msg_rec_counter_acc, global_map_msg_rec_counter)
        
    msg_send_counter_acc /= S
    depth_counter_acc = {k:round(i/S, 3) for k,i in depth_counter_acc.items()}
    map_msg_rec_counter_acc = {k:round(i/S, 3) for k,i in map_msg_rec_counter_acc.items()}
    map_msg_rec_counter_acc = dict(sorted(map_msg_rec_counter_acc.items(), key=lambda item: -item[1]))

    return (msg_send_counter_acc, depth_counter_acc, map_msg_rec_counter_acc)

# Algo Params
S = 1000

# N = 18 # Num nodes
# N = 27 # Num nodes
N = 4 # Num nodes
X = 1/3 # 1st message
Y = 2/3 # 2nd Message
Z = 2/3 # Shrinkage

(msg_count, depth_acc, msg_acc) = run_simulations(S, N, X, Y, Z, True)
test_generator = {}

for k,v in global_map_msg_rec_counter.items():
    test_generator[k] = (global_map_msg_rec_counter[k], global_map_msg_send_counter[k])
for i in range(N):
    k = f"val_{i+1}"
    read = global_map_msg_rec_counter[k]
    write = global_map_msg_send_counter[k]
    print(f"validatorId({i+1}):  {{{read+1}, {write}}},")
# print(test_generator)
# print(global_map_msg_rec_counter)
# print(global_map_msg_send_counter)

print("Simulation results")
print((msg_count, depth_acc, msg_acc))

!!!!!!!! 1.2618595071429148
-----
Msg 1: [ (val_1), **val_2**, val_3, val_4 ]
Msg 2: [ (val_1), val_2, **val_3**, val_4 ]
-----
Msg 2: [ (val_2), **val_3** ]
-----
Msg 2: [ (val_3), **val_4** ]
-----
Msg 2: [ (val_1), **val_2** ]
###################
                 val_1                                
   ┌───────────────┴────┬─────────────────┐           
 val_2                val_1             val_3         
  └───────┐            └───────┐         └───────┐    
        val_3                val_2             val_4  
Target Coverage: 0.6666666666666666
Num nodes: 4
Global Send Counter: 5
Global Set Reached: ['val_1', 'val_2', 'val_3', 'val_4']
Global # Times Received: {'val_2': 1, 'val_3': 1}
Nodes not reached: set()
-----
Msg 1: [ (val_1), **val_2**, val_3, val_4 ]
Msg 2: [ (val_1), val_2, **val_3**, val_4 ]
-----
Msg 2: [ (val_2), **val_3** ]
-----
Msg 2: [ (val_3), **val_4** ]
-----
Msg 2: [ (val_1), **val_2** ]
###################
                 val_1                           

In [1]:
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot
from plotly.subplots import make_subplots

## Compute depth distribution
depth_l = []
for d, c in depth_acc.items():
    depth_l.extend([d] * int(c))
depths, counts = zip(*list(depth_acc.items()))

depth_mean = sum(depth_l) / len(depth_l)
depth_variance = sum([((x - depth_mean) ** 2) for x in depth_l]) / len(depth_l)
depth_std = depth_variance ** 0.5
depth_normal_data = np.random.normal(depth_mean, depth_std, size=100) # replace with your own data source

## Compute send distribution
counts_send = []
for d, c in msg_acc.items():
    counts_send.extend([d] * int(c))
nodes, counts_send = zip(*list(msg_acc.items()))

send_mean = sum(counts_send) / len(counts_send)
send_variance = sum([((x - send_mean) ** 2) for x in counts_send]) / len(counts_send)
send_std = send_variance ** 0.5
send_normal_data = np.random.normal(send_mean, send_std, size=100) # replace with your own data source

# Prepare Figure layout
fig = make_subplots(
    rows=2, cols=1,
    specs=[
        [{"secondary_y": True}],
        [{"secondary_y": True}]],
    row_heights=[5, 5],
    subplot_titles=(
        f'Depth Distribution - Nodes: {N}; Simulations: {S}; Mean: {round(depth_mean, 2)}; Std: {round(depth_std, 2)}',
        f'Message Send Distribution - Nodes: {N}; Simulations: {S}; Mean: {round(send_mean, 2)}; Std: {round(send_std, 2)}',
    ))
fig.update_layout(
    height=800,
)
fig.update_xaxes(range=[0,int(depth_mean * 2)])
fig.update_yaxes(title="Depth counter", secondary_y=False)
fig.update_yaxes(title="Depth counter - simulated normal dist", secondary_y=True)

## Show Raw Data
fig.add_trace(
    go.Bar(x=depths, y=counts, name="depth counter", opacity=0.8),
    secondary_y=False,
)
fig.add_trace(
    go.Bar(x=nodes, y=counts_send, name="message send counter", opacity=0.8),
    secondary_y=False,
    row=2, col=1,
)

## Show Bars - Depths
rstd = depth_mean + depth_std
lstd = depth_mean - depth_std
fig.add_shape(type="line",x0=depth_mean, x1=depth_mean, y0 =0, y1=1 , xref='x', yref='y2',
               line = dict(color = 'darkorange', dash = 'dash'), secondary_y=True)
fig.add_shape(type="line",x0=rstd, x1=rstd, y0 =0, y1=1 , xref='x', yref='y2',
               line = dict(color = 'orangered', dash = 'dash'), secondary_y=True)
fig.add_shape(type="line",x0=lstd, x1=lstd, y0 =0, y1=1 , xref='x', yref='y2',
               line = dict(color = 'orangered', dash = 'dash'), secondary_y=True)

## Show Normal Distribution - Depths
# fig2 = ff.create_distplot([depth_normal_data], ['depth counter - simulated normal dist'])
# fig.add_trace(go.Histogram(fig2['data'][0], marker_color='orange', opacity=0.2), secondary_y=True)
# fig.add_trace(go.Scatter(fig2['data'][1], line=dict(color='orange', width=0.5) ), secondary_y=True)
# fig.add_trace(go.Scatter(fig2['data'][2], line=dict(color='orange', width=0.5)), secondary_y=True)

## Print Results
print(f"Target Coverage: {round(Y, 3)}")
print(f"Num nodes: {N}")
print(f"Num simulations: {S}")
print(f"Avg # messages per simulation: {round(msg_count / S, 2)}")
fig.show()

NameError: name 'depth_acc' is not defined

In [ ]:
def plot(map_to_msg_count, map_to_depth_count, var_name):
    # Prepare Figure layout
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{}, {}]],
        row_heights=[5],
        subplot_titles=(
            f'Avg # Msgs / Node',
            f'Avg Max Depth to Full Coverage',
        ))
    fig.update_layout(
        height=500,
    )
    # fig.update_xaxes(range=[0,int(depth_mean * 2)])
    fig.update_yaxes(title="# Msgs / Node", row=1, col=1)
    fig.update_xaxes(title=var_name, row=1, col=1)

    fig.update_yaxes(title="Max Depth", row=1, col=2)
    fig.update_xaxes(title=var_name, row=1, col=2)

    # ## Show Raw Data
    (x, y) = zip(*dict(map_to_msg_count).items())
    fig.add_trace(
        go.Scatter(x=x, y=y, name="msg count", opacity=0.8),
        row=1, col=1
    )
    (x, y) = zip(*dict(map_to_depth_count).items())
    fig.add_trace(
        go.Scatter(x=x, y=y, name="msg count", opacity=0.8),
        row=1, col=2
    )
    fig.show()

def single_iteration(var, S, N, X, Y, Z, map_to_msg_count, map_to_depth_count):
    try:
        (_, depth_acc, msg_acc) = run_simulations(S, N, X, Y, Z, False)
        # print("Simulation results")
        # print((msg_count, depth_acc, msg_acc))

        # Message Count
        avg_msg = sum(msg_acc.values()) / len(msg_acc)
        map_to_msg_count[var] = avg_msg

        # Depth
        num_total = 0
        denom_total = 0
        for d, c in depth_acc.items():
            num_total += c * d
            denom_total += c
        avg_depth = num_total / denom_total
        map_to_depth_count[var] = avg_depth
    except Exception as e:
        return


In [ ]:

S = 1000
X = 1/3 # 1st message
Y = 2/3 # 2nd Message
Z = 2/3 # Shrinkage

map_to_msg_count = defaultdict()
map_to_depth_count = defaultdict()

for N in range(3, 81):
    single_iteration(N, S, N, X, Y, Z, map_to_msg_count, map_to_depth_count)

plot(map_to_msg_count, map_to_depth_count, "# Nodes")    

!!!!!!!! 1.0
!!!!!!!! 1.2618595071429148
!!!!!!!! 1.4649735207179269
!!!!!!!! 1.6309297535714573
!!!!!!!! 1.7712437491614221
!!!!!!!! 1.892789260714372
!!!!!!!! 2.0
!!!!!!!! 2.095903274289385
!!!!!!!! 2.182658338644138
!!!!!!!! 2.2618595071429146
!!!!!!!! 2.3347175194727927
!!!!!!!! 2.402173502732879
!!!!!!!! 2.464973520717927
!!!!!!!! 2.5237190142858297
!!!!!!!! 2.5789019231625656
!!!!!!!! 2.630929753571457
!!!!!!!! 2.680143859246375
!!!!!!!! 2.7268330278608417
!!!!!!!! 2.771243749161422
!!!!!!!! 2.8135880922155954
!!!!!!!! 2.854049830200271
!!!!!!!! 2.8927892607143724
!!!!!!!! 2.9299470414358537
!!!!!!!! 2.96564727304425
!!!!!!!! 3.0
!!!!!!!! 3.033103256304337
!!!!!!!! 3.0650447521106625
!!!!!!!! 3.0959032742893844
!!!!!!!! 3.1257498572570146
!!!!!!!! 3.154648767857287
!!!!!!!! 3.1826583386441376
!!!!!!!! 3.2098316767340234
!!!!!!!! 3.236217269879349
!!!!!!!! 3.2618595071429146
!!!!!!!! 3.2867991282182727
!!!!!!!! 3.3110736128178324
!!!!!!!! 3.3347175194727923
!!!!!!!! 3.357762781432

In [ ]:
import numpy as np

S = 1000
N = 27
X = 1/3 # 1st message
Y = 3/4 # 2nd Message
Z = 2/3 # Shrinkage

map_to_msg_count = defaultdict()
map_to_depth_count = defaultdict()

for X in np.arange(0.1, Y, 0.01):
    print("X:", X)
    single_iteration(X, S, N, X, Y, Z, map_to_msg_count, map_to_depth_count)

plot(map_to_msg_count, map_to_depth_count, "X target %")

X: 0.1
!!!!!!!! 3.0
X: 0.11
!!!!!!!! 3.0
X: 0.12
!!!!!!!! 3.0
X: 0.13
!!!!!!!! 3.0
X: 0.13999999999999999
!!!!!!!! 3.0
X: 0.14999999999999997
!!!!!!!! 3.0
X: 0.15999999999999998
!!!!!!!! 3.0
X: 0.16999999999999998
!!!!!!!! 3.0
X: 0.17999999999999997
!!!!!!!! 3.0
X: 0.18999999999999995
!!!!!!!! 3.0
X: 0.19999999999999996
!!!!!!!! 3.0
X: 0.20999999999999996
!!!!!!!! 3.0
X: 0.21999999999999995
!!!!!!!! 3.0
X: 0.22999999999999995
!!!!!!!! 3.0
X: 0.23999999999999994
!!!!!!!! 3.0
X: 0.24999999999999992
!!!!!!!! 3.0
X: 0.2599999999999999
!!!!!!!! 3.0
X: 0.2699999999999999
!!!!!!!! 3.0
X: 0.2799999999999999
!!!!!!!! 3.0
X: 0.2899999999999999
!!!!!!!! 3.0
X: 0.29999999999999993
!!!!!!!! 3.0
X: 0.30999999999999994
!!!!!!!! 3.0
X: 0.3199999999999999
!!!!!!!! 3.0
X: 0.32999999999999985
!!!!!!!! 3.0
X: 0.33999999999999986
!!!!!!!! 3.0
X: 0.34999999999999987
!!!!!!!! 3.0
X: 0.3599999999999999
!!!!!!!! 3.0
X: 0.3699999999999999
!!!!!!!! 3.0
X: 0.3799999999999999
!!!!!!!! 3.0
X: 0.3899999999999999
!!!

In [ ]:
import numpy as np

S = 1000
N = 27
X = 0.25 # 1st message
Y = 0.75 # 2nd Message

map_to_msg_count = defaultdict()
map_to_depth_count = defaultdict()

for Z in np.arange(0.1, 0.9, 0.01):
    print("Z:", Z)
    single_iteration(Z, S, N, X, Y, Z, map_to_msg_count, map_to_depth_count)

plot(map_to_msg_count, map_to_depth_count, "Shrinkage %")

Z: 0.1
!!!!!!!! 3.0
Z: 0.11
!!!!!!!! 3.0
Z: 0.12
!!!!!!!! 3.0
Z: 0.13
!!!!!!!! 3.0
Z: 0.13999999999999999
!!!!!!!! 3.0
Z: 0.14999999999999997
!!!!!!!! 3.0
Z: 0.15999999999999998
!!!!!!!! 3.0
Z: 0.16999999999999998
!!!!!!!! 3.0
Z: 0.17999999999999997
!!!!!!!! 3.0
Z: 0.18999999999999995
!!!!!!!! 3.0
Z: 0.19999999999999996
!!!!!!!! 3.0
Z: 0.20999999999999996
!!!!!!!! 3.0
Z: 0.21999999999999995
!!!!!!!! 3.0
Z: 0.22999999999999995
!!!!!!!! 3.0
Z: 0.23999999999999994
!!!!!!!! 3.0
Z: 0.24999999999999992
!!!!!!!! 3.0
Z: 0.2599999999999999
!!!!!!!! 3.0
Z: 0.2699999999999999
!!!!!!!! 3.0
Z: 0.2799999999999999
!!!!!!!! 3.0
Z: 0.2899999999999999
!!!!!!!! 3.0
Z: 0.29999999999999993
!!!!!!!! 3.0
Z: 0.30999999999999994
!!!!!!!! 3.0
Z: 0.3199999999999999
!!!!!!!! 3.0
Z: 0.32999999999999985
!!!!!!!! 3.0
Z: 0.33999999999999986
!!!!!!!! 3.0
Z: 0.34999999999999987
!!!!!!!! 3.0
Z: 0.3599999999999999
!!!!!!!! 3.0
Z: 0.3699999999999999
!!!!!!!! 3.0
Z: 0.3799999999999999
!!!!!!!! 3.0
Z: 0.3899999999999999
!!!

In [ ]:
import numpy as np

S = 1000
N = 27
X = 0.01 # 1st message
Z = 2/3 # Shrinkage

map_to_msg_count = defaultdict()
map_to_depth_count = defaultdict()

for Y in np.arange(X, 
0.9, 0.01):
    print("Y:", Y)
    single_iteration(Y, S, N, X, Y, Z, map_to_msg_count, map_to_depth_count)

plot(map_to_msg_count, map_to_depth_count, "Second Target %")

Y: 0.01
!!!!!!!! 3.0
Y: 0.02
!!!!!!!! 3.0
Y: 0.03
!!!!!!!! 3.0
Y: 0.04
!!!!!!!! 3.0
Y: 0.05
!!!!!!!! 3.0
Y: 0.060000000000000005
!!!!!!!! 3.0
Y: 0.06999999999999999
!!!!!!!! 3.0
Y: 0.08
!!!!!!!! 3.0
Y: 0.09
!!!!!!!! 3.0
Y: 0.09999999999999999
!!!!!!!! 3.0
Y: 0.11
!!!!!!!! 3.0
Y: 0.12
!!!!!!!! 3.0
Y: 0.13
!!!!!!!! 3.0
Y: 0.14
!!!!!!!! 3.0
Y: 0.15000000000000002
!!!!!!!! 3.0
Y: 0.16
!!!!!!!! 3.0
Y: 0.17
!!!!!!!! 3.0
Y: 0.18000000000000002
!!!!!!!! 3.0
Y: 0.19
!!!!!!!! 3.0
Y: 0.2
!!!!!!!! 3.0
Y: 0.21000000000000002
!!!!!!!! 3.0
Y: 0.22
!!!!!!!! 3.0
Y: 0.23
!!!!!!!! 3.0
Y: 0.24000000000000002
!!!!!!!! 3.0
Y: 0.25
!!!!!!!! 3.0
Y: 0.26
!!!!!!!! 3.0
Y: 0.27
!!!!!!!! 3.0
Y: 0.28
!!!!!!!! 3.0
Y: 0.29000000000000004
!!!!!!!! 3.0
Y: 0.3
!!!!!!!! 3.0
Y: 0.31
!!!!!!!! 3.0
Y: 0.32
!!!!!!!! 3.0
Y: 0.33
!!!!!!!! 3.0
Y: 0.34
!!!!!!!! 3.0
Y: 0.35000000000000003
!!!!!!!! 3.0
Y: 0.36000000000000004
!!!!!!!! 3.0
Y: 0.37
!!!!!!!! 3.0
Y: 0.38
!!!!!!!! 3.0
Y: 0.39
!!!!!!!! 3.0
Y: 0.4
!!!!!!!! 3.0
Y: 0.4100000